## Project: Sign Language Recognition System
- [Introduction](#intro)
- [Part 1 Feature Selection](#part1_tutorial)
    - [Tutorial](#part1_tutorial)
    - [Features Submission](#part1_submission)
    - [Features Unittest](#part1_test)
- [Part 2 Train the models](#part2_tutorial)
    - [Tutorial](#part2_tutorial)
    - [Model Selection Score Submission](#part2_submission)
    - [Model Score Unittest](#part2_test)
- [Part 3 Build a Recognizer](#part3_tutorial)
    - [Tutorial](#part3_tutorial)
    - [Recognizer Submission](#part3_submission)
    - [Recognizer Unittest](#part3_test)
- [Part 4 Improve the WER with Language Models](#part4_info)

<a id='intro'></a>
## Introduction
The overall goal of this project is to build a word recognizer for American Sign Language video sequences, demonstrating the power of probabalistic models.  In particular, this project employs  [hidden Markov models (HMM's)](https://en.wikipedia.org/wiki/Hidden_Markov_model) to analyze a series of measurements taken from videos of American Sign Language (ASL) collected for research (see the [RWTH-BOSTON-104 Database](http://www-i6.informatik.rwth-aachen.de/~dreuw/database-rwth-boston-104.php)).  In this video, the right-hand x and y locations are plotted as the speaker signs the sentence.
[![ASLR demo](http://www-i6.informatik.rwth-aachen.de/~dreuw/images/demosample.png)](https://drive.google.com/open?id=0B_5qGuFe-wbhUXRuVnNZVnMtam8)

The raw data, train, and test sets are pre-defined. Derived a variety of feature sets (explored in Part 1), as well as implemented three different model selection criterion to determine the optimal number of hidden states for each word model (explored in Part 2). Finally, Part 3 implemented the recognizer and compare the effects the different combinations of feature sets and model selection criteria.  

<a id='part1_tutorial'></a>
## PART 1: Data

### Features Tutorial
##### Load the initial database
A data handler designed for this database is provided in the student codebase as the `AslDb` class in the `asl_data` module.  This handler creates the initial [pandas](http://pandas.pydata.org/pandas-docs/stable/) dataframe from the corpus of data included in the `data` directory as well as dictionaries suitable for extracting data in a format friendly to the [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/) library.  We'll use those to create models in Part 2.

In [1]:
import numpy as np
import pandas as pd
from asl_data import AslDb


asl = AslDb() # initializes the database
asl.df.head() # displays the first five rows of the asl database, indexed by video and frame

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker
video frame                                                           
98    0         149     181      170      175     161      62  woman-1
      1         149     181      170      175     161      62  woman-1
      2         149     181      170      175     161      62  woman-1
      3         149     181      170      175     161      62  woman-1
      4         149     181      170      175     161      62  woman-1

In [2]:
asl.df.ix[98,1]  # look at the data available for an individual frame

left-x         149
left-y         181
right-x        170
right-y        175
nose-x         161
nose-y          62
speaker    woman-1
Name: (98, 1), dtype: object

The frame represented by video 98, frame 1 is shown here:
![Video 98](http://www-i6.informatik.rwth-aachen.de/~dreuw/database/rwth-boston-104/overview/images/orig/098-start.jpg)

##### Feature selection for training the model
The objective of feature selection when training a model is to choose the most relevant variables while keeping the model as simple as possible, thus reducing training time.  We can use the raw features already provided or derive our own and add columns to the pandas dataframe `asl.df` for selection. As an example, in the next cell a feature named `'grnd-ry'` is added. This feature is the difference between the right-hand y value and the nose y value, which serves as the "ground" right y value. 

In [3]:
asl.df['grnd-ry'] = asl.df['right-y'] - asl.df['nose-y']
asl.df.head()  # the new feature 'grnd-ry' is now in the frames dictionary

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  
video frame           
98    0          113  
      1          113  
      2          113  
      3          113  
      4          113

In [5]:
# collect the features into a list
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']
 #show a single set of features for a given (video, frame) tuple
[asl.df.ix[98,1][v] for v in features_ground]

[9, 113, -12, 119]

##### Build the training set
Now that we have a feature list defined, we can pass that list to the `build_training` method to collect the features for all the words in the training set.  Each word in the training set has multiple examples from various videos.  Below we can see the unique words that have been loaded into the training set:

In [6]:
training = asl.build_training(features_ground)
print("Training words: {}".format(training.words))

Training words: ['JOHN', 'WRITE', 'HOMEWORK', 'IX-1P', 'SEE', 'YESTERDAY', 'IX', 'LOVE', 'MARY', 'CAN', 'GO', 'GO1', 'FUTURE', 'GO2', 'PARTY', 'FUTURE1', 'HIT', 'BLAME', 'FRED', 'FISH', 'WONT', 'EAT', 'BUT', 'CHICKEN', 'VEGETABLE', 'CHINA', 'PEOPLE', 'PREFER', 'BROCCOLI', 'LIKE', 'LEAVE', 'SAY', 'BUY', 'HOUSE', 'KNOW', 'CORN', 'CORN1', 'THINK', 'NOT', 'PAST', 'LIVE', 'CHICAGO', 'CAR', 'SHOULD', 'DECIDE', 'VISIT', 'MOVIE', 'WANT', 'SELL', 'TOMORROW', 'NEXT-WEEK', 'NEW-YORK', 'LAST-WEEK', 'WILL', 'FINISH', 'ANN', 'READ', 'BOOK', 'CHOCOLATE', 'FIND', 'SOMETHING-ONE', 'POSS', 'BROTHER', 'ARRIVE', 'HERE', 'GIVE', 'MAN', 'NEW', 'COAT', 'WOMAN', 'GIVE1', 'HAVE', 'FRANK', 'BREAK-DOWN', 'SEARCH-FOR', 'WHO', 'WHAT', 'LEG', 'FRIEND', 'CANDY', 'BLUE', 'SUE', 'BUY1', 'STOLEN', 'OLD', 'STUDENT', 'VIDEOTAPE', 'BORROW', 'MOTHER', 'POTATO', 'TELL', 'BILL', 'THROW', 'APPLE', 'NAME', 'SHOOT', 'SAY-1P', 'SELF', 'GROUP', 'JANA', 'TOY1', 'MANY', 'TOY', 'ALL', 'BOY', 'TEACHER', 'GIRL', 'BOX', 'GIVE2', 'GIVE3

The training data in `training` is an object of class `WordsData` defined in the `asl_data` module.  in addition to the `words` list, data can be accessed with the `get_all_sequences`, `get_all_Xlengths`, `get_word_sequences`, and `get_word_Xlengths` methods. We need the `get_word_Xlengths` method to train multiple sequences with the `hmmlearn` library.  In the following example, notice that there are two lists; the first is a concatenation of all the sequences(the X portion) and the second is a list of the sequence lengths(the Lengths portion).

###### More feature sets
So far we have a simple feature set that is enough to get started modeling.  However, we might get better results if we manipulate the raw values a bit more, so we will go ahead and set up some other options now for experimentation later.  For example, we could normalize each speaker's range of motion with grouped statistics using [Pandas stats](http://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats) functions and [pandas groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html).  Below is an example for finding the means of all speaker subgroups.

In [9]:
df_means = asl.df.groupby('speaker').mean()
df_means

,left-x,left-y,right-x,right-y,nose-x,nose-y,grnd-ry,grnd-rx,grnd-ly,grnd-lx
speaker,,,,,,,,,,
man-1,206.248203,218.679449,155.464350,150.371031,175.031756,61.642600,88.728430,-19.567406,157.036848,31.216447
woman-1,164.661438,161.271242,151.017865,117.332462,162.655120,57.245098,60.087364,-11.637255,104.026144,2.006318
woman-2,183.214509,176.527232,156.866295,119.835714,170.318973,58.022098,61.813616,-13.452679,118.505134,12.895536


To select a mean that matches by speaker, use the pandas [map](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) method:

In [10]:
asl.df['left-x-mean']= asl.df['speaker'].map(df_means['left-x'])
asl.df.head()

left-x  left-y  right-x  right-y  nose-x  nose-y  speaker  \
video frame                                                              
98    0         149     181      170      175     161      62  woman-1   
      1         149     181      170      175     161      62  woman-1   
      2         149     181      170      175     161      62  woman-1   
      3         149     181      170      175     161      62  woman-1   
      4         149     181      170      175     161      62  woman-1   

             grnd-ry  grnd-rx  grnd-ly  grnd-lx  left-x-mean  
video frame                                                   
98    0          113        9      119      -12   164.661438  
      1          113        9      119      -12   164.661438  
      2          113        9      119      -12   164.661438  
      3          113        9      119      -12   164.661438  
      4          113        9      119      -12   164.661438

<a id='part1_submission'></a>
### Features Implementation Submission
Implement four feature sets and answer the question that follows.
- normalized Cartesian coordinates
    - use *mean* and *standard deviation* statistics and the [standard score](https://en.wikipedia.org/wiki/Standard_score) equation to account for speakers with different heights and arm length
    
- polar coordinates
    - calculate polar coordinates with [Cartesian to polar equations](https://en.wikipedia.org/wiki/Polar_coordinate_system#Converting_between_polar_and_Cartesian_coordinates)
    - use the [np.arctan2](https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.arctan2.html) function and *swap the x and y axes* to move the $0$ to $2\pi$ discontinuity to 12 o'clock instead of 3 o'clock;  in other words, the normal break in radians value from $0$ to $2\pi$ occurs directly to the left of the speaker's nose, which may be in the signing area and interfere with results.  By swapping the x and y axes, that discontinuity move to directly above the speaker's head, an area not generally used in signing.

- delta difference
    - as described in Thad's lecture, use the difference in values between one frame and the next frames as features
    - pandas [diff method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.diff.html) and [fillna method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) will be helpful for this one

- custom features
    - These are your own design; combine techniques used above or come up with something else entirely. We look forward to seeing what you come up with! 
    Some ideas to get you started:
        - normalize using a [feature scaling equation](https://en.wikipedia.org/wiki/Feature_scaling)
        - normalize the polar coordinates
        - adding additional deltas


In [12]:
# add features for normalized by speaker values of left, right, x, y
# Name these 'norm-rx', 'norm-ry', 'norm-lx', and 'norm-ly'
# using Z-score scaling (X-Xmean)/Xstd

features_norm = ['norm-rx', 'norm-ry', 'norm-lx','norm-ly']

asl.df['left-y-mean']= asl.df['speaker'].map(df_means['left-y'])
asl.df['right-x-mean']= asl.df['speaker'].map(df_means['right-x'])
asl.df['right-y-mean']= asl.df['speaker'].map(df_means['right-y'])

asl.df['left-x-std']= asl.df['speaker'].map(df_std['left-x'])
asl.df['left-y-std']= asl.df['speaker'].map(df_std['left-y'])
asl.df['right-x-std']= asl.df['speaker'].map(df_std['right-x'])
asl.df['right-y-std']= asl.df['speaker'].map(df_std['right-y'])

asl.df['norm-rx']=(asl.df['right-x']-asl.df['right-x-mean'])/asl.df['right-x-std']
asl.df['norm-ry']=(asl.df['right-y']-asl.df['right-y-mean'])/asl.df['right-y-std']
asl.df['norm-lx']=(asl.df['left-x']-asl.df['left-x-mean'])/asl.df['left-x-std']
asl.df['norm-ly']=(asl.df['left-y']-asl.df['left-y-mean'])/asl.df['left-y-std']

In [31]:
asl.df.fillna(value=0, inplace=True)

In [17]:
# add features for polar coordinate values where the nose is the origin
# Name these 'polar-rr', 'polar-rtheta', 'polar-lr', and 'polar-ltheta'
# Note that 'polar-rr' and 'polar-rtheta' refer to the radius and angle

features_polar = ['polar-rr', 'polar-rtheta', 'polar-lr', 'polar-ltheta']

In [18]:
asl.df['polar-ltheta'] = np.arctan2(asl.df['grnd-lx'], asl.df['grnd-ly'])

In [19]:
asl.df['polar-rtheta'] = np.arctan2(asl.df['grnd-rx'], asl.df['grnd-ry'])

In [20]:
asl.df['polar-lr'] = np.sqrt(asl.df['grnd-lx']**2 + asl.df['grnd-ly']**2)

In [21]:
asl.df['polar-rr'] = np.sqrt(asl.df['grnd-rx']**2 + asl.df['grnd-ry']**2)

In [22]:
# add features for left, right, x, y differences by one time step, i.e. the "delta" values discussed in the lecture
# Name these 'delta-rx', 'delta-ry', 'delta-lx', and 'delta-ly'

features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']

In [23]:
asl.df['delta-rx'] = asl.df['right-x'].diff()
asl.df['delta-ry'] = asl.df['right-y'].diff()
asl.df['delta-lx'] = asl.df['left-x'].diff()
asl.df['delta-ly'] = asl.df['left-y'].diff()

In [26]:
# define a list named 'features_custom' for building the training set

features_custom = ['norm-p-rtheta', 'norm-p-rr', 'norm-p-ltheta', 'norm-p-lr']

In [27]:
asl.df['norm-p-rtheta'] = np.arctan2((asl.df['grnd-rx']-asl.df['speaker'].map(df_means['grnd-rx']))
                                     /asl.df['speaker'].map(df_std['grnd-rx']),
                                     (asl.df['grnd-ry']-asl.df['speaker'].map(df_means['grnd-ry']))
                                     /asl.df['speaker'].map(df_std['grnd-ry']))
asl.df['norm-p-ltheta'] = np.arctan2((asl.df['grnd-lx']-asl.df['speaker'].map(df_means['grnd-lx']))
                                     /asl.df['speaker'].map(df_std['grnd-rx']),
                                     (asl.df['grnd-ly']-asl.df['speaker'].map(df_means['grnd-ly']))
                                     /asl.df['speaker'].map(df_std['grnd-ly']))

In [28]:
asl.df['norm-p-rr'] = np.sqrt(((asl.df['grnd-rx']-asl.df['speaker'].map(df_means['grnd-rx']))
                                     /asl.df['speaker'].map(df_std['grnd-rx']))**2 + 
                              ((asl.df['grnd-ry']-asl.df['speaker'].map(df_means['grnd-ry']))
                                     /asl.df['speaker'].map(df_std['grnd-ry']))**2)
asl.df['norm-p-lr'] = np.sqrt(((asl.df['grnd-lx']-asl.df['speaker'].map(df_means['grnd-lx']))
                                     /asl.df['speaker'].map(df_std['grnd-lx']))**2 + 
                              ((asl.df['grnd-ly']-asl.df['speaker'].map(df_means['grnd-ly']))
                                     /asl.df['speaker'].map(df_std['grnd-ly']))**2)

**Question 1:**  What custom features did you choose for the features_custom set and why?

**Answer 1:** I converted normalized cartesian features_norm into normalized polar features. Now the choices of features have normalized/unnormalized, cartesian/polar coordinates, which exhausts the possibilities. The normalization also reduces error due to variance from different people.

<a id='part2_tutorial'></a>
## PART 2: Model Selection
### Model Selection Tutorial
The objective of Model Selection is to tune the number of states for each word HMM prior to testing on unseen data.  In this section you will explore three methods: 
- Log likelihood using cross-validation folds (CV)
- Bayesian Information Criterion (BIC)
- Discriminative Information Criterion (DIC) 

##### Train a single word
Now that we have built a training set with sequence data, we can "train" models for each word.  As a simple starting example, we train a single word using Gaussian hidden Markov models (HMM).   By using the `fit` method during training, the [Baum-Welch Expectation-Maximization](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) (EM) algorithm is invoked iteratively to find the best estimate for the model *for the number of hidden states specified* from a group of sample seequences. For this example, we *assume* the correct number of hidden states is 3, but that is just a guess.  How do we know what the "best" number of states for training is?  We will need to find some model selection technique to choose the best parameter.

In [33]:
import warnings
from hmmlearn.hmm import GaussianHMM

def train_a_word(word, num_hidden_states, features):
    
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    training = asl.build_training(features)  
    X, lengths = training.get_word_Xlengths(word)
    model = GaussianHMM(n_components=num_hidden_states, n_iter=1000).fit(X, lengths)
    logL = model.score(X, lengths)
    return model, logL

demoword = 'BOOK'
model, logL = train_a_word(demoword, 3, features_ground)
print("Number of states trained in model for {} is {}".format(demoword, model.n_components))
print("logL = {}".format(logL))

Number of states trained in model for BOOK is 3
logL = -2331.1138127433205


The HMM model has been trained and information can be pulled from the model, including means and variances for each feature and hidden state.  The [log likelihood](http://math.stackexchange.com/questions/892832/why-we-consider-log-likelihood-instead-of-likelihood-in-gaussian-distribution) for any individual sample or group of samples can also be calculated with the `score` method.

In [34]:
def show_model_stats(word, model):
    print("Number of states trained in model for {} is {}".format(word, model.n_components))    
    variance=np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])    
    for i in range(model.n_components):  # for each hidden state
        print("hidden state #{}".format(i))
        print("mean = ", model.means_[i])
        print("variance = ", variance[i])
        print()
    
show_model_stats(demoword, model)

Number of states trained in model for BOOK is 3
hidden state #0
mean =  [ -11.45300909   94.109178     19.03512475  102.2030162 ]
variance =  [  77.403668    203.35441965   26.68898447  156.12444034]

hidden state #1
mean =  [ -3.46504869  50.66686933  14.02391587  52.04731066]
variance =  [ 49.12346305  43.04799144  39.35109609  47.24195772]

hidden state #2
mean =  [ -1.12415027  69.44164191  17.02866283  77.7231196 ]
variance =  [ 19.70434594  16.83041492  30.51552305  11.03678246]



In [35]:
my_testword = 'CHOCOLATE'
model_ground, logL = train_a_word(my_testword, 3, features_ground) # Experiment here with different parameters
show_model_stats(my_testword, model_ground)
print("logL = {}".format(logL))

Number of states trained in model for CHOCOLATE is 3
hidden state #0
mean =  [ -9.30211403  55.32333876   6.92259936  71.24057775]
variance =  [ 16.16920957  46.50917372   3.81388185  15.79446427]

hidden state #1
mean =  [   0.58333333   87.91666667   12.75        108.5       ]
variance =  [  39.41055556   18.74388889    9.855       144.4175    ]

hidden state #2
mean =  [ -5.40587658  60.1652424    2.32479599  91.3095432 ]
variance =  [   7.95073876   64.13103127   13.68077479  129.5912395 ]

logL = -601.3291470028619


#####  ModelSelector class
Review the `ModelSelector` class from the codebase found in the `my_model_selectors.py` module.  It is designed to be a strategy pattern for choosing different model selectors.  For the project submission in this section, subclass `SelectorModel` to implement the following model selectors.  In other words, you will write your own classes/functions in the `my_model_selectors.py` module and run them from this notebook:

- `SelectorCV `:  Log likelihood with CV
- `SelectorBIC`: BIC 
- `SelectorDIC`: DIC

You will train each word in the training set with a range of values for the number of hidden states, and then score these alternatives with the model selector, choosing the "best" according to each strategy. The simple case of training with a constant value for `n_components` can be called using the provided `SelectorConstant` subclass as follow:

In [46]:
from my_model_selectors import SelectorConstant

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
word = 'VEGETABLE' # Experiment here with different words
model_g = SelectorConstant(training.get_all_sequences(), training.get_all_Xlengths(), word, n_constant=3).select()
print("Number of states trained in model for {} is {}".format(word, model_g.n_components))

Number of states trained in model for VEGETABLE is 3


In [47]:
training = asl.build_training(features_norm)
model_n = SelectorConstant(training.get_all_sequences(), training.get_all_Xlengths(), word, n_constant=3).select()
print("Number of states trained in model for {} is {}".format(word, model_n.n_components))

Number of states trained in model for VEGETABLE is 3


In [48]:
training = asl.build_training(features_norm)
word = 'JOHN'
model_n = SelectorConstant(training.get_all_sequences(), training.get_all_Xlengths(), word, n_constant=3).select()
print("Number of states trained in model for {} is {}".format(word, model_n.n_components))

Number of states trained in model for JOHN is 3


##### Cross-validation folds
If we simply score the model with the Log Likelihood calculated from the feature sequences it has been trained on, we should expect that more complex models will have higher likelihoods. However, that doesn't tell us which would have a better likelihood score on unseen data.  The model will likely be overfit as complexity is added.  To estimate which topology model is better using only the training data, we can compare scores using cross-validation.  One technique for cross-validation is to break the training set into "folds" and rotate which fold is left out of training.  The "left out" fold scored.  This gives us a proxy method of finding the best model to use on "unseen data". In the following example, a set of word sequences is broken into three folds using the [scikit-learn Kfold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) class object. When you implement `SelectorCV`, you will use this technique.

In [49]:
from sklearn.model_selection import KFold

training = asl.build_training(features_ground) # Experiment here with different feature sets
word = 'VEGETABLE' # Experiment here with different words
word_sequences = training.get_word_sequences(word)
split_method = KFold()
for cv_train_idx, cv_test_idx in split_method.split(word_sequences):
    print("Train fold indices:{} Test fold indices:{}".format(cv_train_idx, cv_test_idx))  # view indices of the folds

Train fold indices:[2 3 4 5] Test fold indices:[0 1]
Train fold indices:[0 1 4 5] Test fold indices:[2 3]
Train fold indices:[0 1 2 3] Test fold indices:[4 5]


In [50]:
len(word_sequences)

6

**Tip:** In order to run `hmmlearn` training using the X,lengths tuples on the new folds, subsets must be combined based on the indices given for the folds.  A helper utility has been provided in the `asl_utils` module named `combine_sequences` for this purpose.

##### Scoring models with other criterion
Scoring model topologies with **BIC** balances fit and complexity within the training set for each word.  In the BIC equation, a penalty term penalizes complexity to avoid overfitting, so that it is not necessary to also use cross-validation in the selection process.  There are a number of references on the internet for this criterion.  These [slides](http://www2.imm.dtu.dk/courses/02433/doc/ch6_slides.pdf) include a formula you may find helpful for your implementation.

The advantages of scoring model topologies with **DIC** over BIC are presented by Alain Biem in this [reference](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.6208&rep=rep1&type=pdf) (also found [here](https://pdfs.semanticscholar.org/ed3d/7c4a5f607201f3848d4c02dd9ba17c791fc2.pdf)).  DIC scores the discriminant ability of a training set for one word against competing words.  Instead of a penalty term for complexity, it provides a penalty if model liklihoods for non-matching words are too similar to model likelihoods for the correct word in the word set.

<a id='part2_submission'></a>
### Model Selection Implementation Submission
Implement `SelectorCV`, `SelectorBIC`, and `SelectorDIC` classes in the `my_model_selectors.py` module.  Run the selectors on the following five words. Then answer the questions about your results.

**Tip:** The `hmmlearn` library may not be able to train or score all models.  Implement try/except contructs as necessary to eliminate non-viable models from consideration.

In [51]:
words_to_train = ['FISH', 'BOOK', 'VEGETABLE', 'FUTURE', 'JOHN']
import timeit

In [52]:
from importlib import reload

In [53]:
# Implement SelectorCV in my_model_selector.py

import my_model_selectors
reload(my_model_selectors)

from my_model_selectors import SelectorCV

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorCV(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

Training complete for FISH with 3 states with time 0.022902660002728226 seconds
Training complete for BOOK with 6 states with time 6.945106381997903 seconds
Training complete for VEGETABLE with 2 states with time 3.2236466169997584 seconds
Training complete for FUTURE with 2 states with time 6.541952384999604 seconds
Training complete for JOHN with 12 states with time 69.74849759000062 seconds


In [54]:
# Implement SelectorBIC in module my_model_selectors.py
reload(my_model_selectors)
from my_model_selectors import SelectorBIC

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorBIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

Training complete for FISH with 5 states with time 0.5159769660021993 seconds
Training complete for BOOK with 8 states with time 3.57981252099853 seconds
Training complete for VEGETABLE with 15 states with time 1.1369084480029414 seconds
Training complete for FUTURE with 9 states with time 4.029095445999701 seconds
Training complete for JOHN with 14 states with time 36.908201174999704 seconds


In [55]:
# Implement SelectorDIC in module my_model_selectors.py
reload(my_model_selectors)
from my_model_selectors import SelectorDIC

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorDIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

Training complete for FISH with 3 states with time 1.3569400329943164 seconds
Training complete for BOOK with 15 states with time 7.680827174001024 seconds
Training complete for VEGETABLE with 15 states with time 6.634045646998857 seconds
Training complete for FUTURE with 15 states with time 7.720871780999005 seconds
Training complete for JOHN with 15 states with time 41.79895381700044 seconds


In [143]:
reload(my_model_selectors)

<module 'my_model_selectors' from '/home/gang/Documents/AI/AIND-Recognizer/my_model_selectors.py'>

**Question 2:**  Compare and contrast the possible advantages and disadvantages of the various model selectors implemented.

**Answer 2:** CV tends to give less number of states in the final model, which is more desirable. The flip side is that it may take longer time to train when the number of optimal states are large. BIC performs slightly better than DIC, since there are four 15-states (boundary max value) final results from training in DIC which may indicate it is not converged yet. 

<a id='part3_tutorial'></a>
## PART 3: Recognizer
The objective of this section is to "put it all together".  Using the four feature sets created and the three model selectors, you will experiment with the models and present your results.  Instead of training only five specific words as in the previous section, train the entire set with a feature set and model selector strategy.  
### Recognizer Tutorial
##### Train the full training set
The following example trains the entire set with the example `features_ground` and `SelectorConstant` features and model selector.  Use this pattern for you experimentation and final submission cells.



In [57]:
# autoreload for automatically reloading changes made in my_model_selectors and my_recognizer
%load_ext autoreload
%autoreload 2

from my_model_selectors import SelectorConstant

def train_all_words(features, model_selector):
    training = asl.build_training(features)  # Experiment here with different feature sets defined in part 1
    sequences = training.get_all_sequences()
    Xlengths = training.get_all_Xlengths()
    model_dict = {}
    for word in training.words:
        model = model_selector(sequences, Xlengths, word, 
                        n_constant=3).select()
        model_dict[word]=model
    return model_dict

models = train_all_words(features_ground, SelectorConstant)
print("Number of word models returned = {}".format(len(models)))

Number of word models returned = 112


##### Load the test set
The `build_test` method in `ASLdb` is similar to the `build_training` method already presented, but there are a few differences:
- the object is type `SinglesData` 
- the internal dictionary keys are the index of the test word rather than the word itself
- the getter methods are `get_all_sequences`, `get_all_Xlengths`, `get_item_sequences` and `get_item_Xlengths`

In [58]:
test_set = asl.build_test(features_ground)
print("Number of test set items: {}".format(test_set.num_items))
print("Number of test set sentences: {}".format(len(test_set.sentences_index)))

Number of test set items: 178
Number of test set sentences: 40


<a id='part3_submission'></a>
### Recognizer Implementation 
Implemented a recognizer following guidance in the `my_recognizer.py` module.  Experiment with the four feature sets and the three model selection methods (that's 12 possible combinations). Provided code cells of **only three** interesting combinations for discussion (see questions below). 

In [249]:
# implement the recognize method in my_recognizer
%load_ext autoreload
%autoreload 2

from my_recognizer import recognize
from asl_utils import show_errors, show_errors_2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [258]:
# 2 = polar * CV
# Choose a feature set and model selector
features = features_polar # change as needed
model_selector = SelectorCV # change as needed

# Recognize the test set and display the result with the show_errors method
start = timeit.default_timer()

models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

end = timeit.default_timer()-start

print("Scoring complete with time {} seconds".format(end))


**** WER = 0.5561797752808989
Total correct: 79 out of 178
Video  Recognized                                                    Correct
    2: *FRANK WRITE *NEW                                             JOHN WRITE HOMEWORK
    7: JOHN *HAVE *IX *WHAT                                          JOHN CAN GO CAN
   12: *IX CAN *WHAT CAN                                             JOHN CAN GO CAN
   21: JOHN *HOMEWORK *NEW *PREFER *CAR *CAR EAT *TOMORROW           JOHN FISH WONT EAT BUT CAN EAT CHICKEN
   25: JOHN *IX IX IX IX                                             JOHN LIKE IX IX IX
   28: JOHN *WHO IX IX *LOVE                                         JOHN LIKE IX IX IX
   30: JOHN LIKE IX *MARY IX                                         JOHN LIKE IX IX IX
   36: *JOHN *EAT *GIRL *GIVE *MARY *MARY                            MARY VEGETABLE KNOW IX LIKE CORN1
   40: JOHN *GIVE *CORN *JOHN *IX                                    JOHN IX THINK MARY LOVE
   43: JOHN *SHOULD BUY HOUSE       

In [269]:
# 9 = polar * BIC
# Choose a feature set and model selector
features = features_polar # change as needed
model_selector = SelectorBIC # change as needed

# Recognize the test set and display the result with the show_errors method
start = timeit.default_timer()

models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

end = timeit.default_timer()-start

print("Scoring complete with time {} seconds".format(end))


**** WER = 0.5617977528089888
Total correct: 78 out of 178
Video  Recognized                                                    Correct
    2: *GO *BROTHER *GIVE1                                           JOHN WRITE HOMEWORK
    7: JOHN *BOX GO *ARRIVE                                          JOHN CAN GO CAN
   12: JOHN *WHAT *JOHN CAN                                          JOHN CAN GO CAN
   21: JOHN *ARRIVE WONT *PREFER *GIVE1 *WHAT *FUTURE *WHO           JOHN FISH WONT EAT BUT CAN EAT CHICKEN
   25: JOHN LIKE IX *WHO IX                                          JOHN LIKE IX IX IX
   28: JOHN *FUTURE IX *FUTURE *LOVE                                 JOHN LIKE IX IX IX
   30: JOHN LIKE *MARY *MARY *MARY                                   JOHN LIKE IX IX IX
   36: *IX *VISIT *GIVE *GIVE *MARY *MARY                            MARY VEGETABLE KNOW IX LIKE CORN1
   40: JOHN *GO *GIVE *JOHN *MARY                                    JOHN IX THINK MARY LOVE
   43: JOHN *JOHN BUY HOUSE         

In [60]:
# 10 = norm * DIC
# Choose a feature set and model selector
features = features_norm # change as needed
model_selector = SelectorDIC # change as needed

# Recognize the test set and display the result with the show_errors method
start = timeit.default_timer()

models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

end = timeit.default_timer()-start

print("Scoring complete with time {} seconds".format(end))


**** WER = 0.5955056179775281
Total correct: 72 out of 178
Video  Recognized                                                    Correct
    2: JOHN WRITE *ARRIVE                                            JOHN WRITE HOMEWORK
    7: *MARY *CAR GO CAN                                             JOHN CAN GO CAN
   12: JOHN *WHAT *ARRIVE CAN                                        JOHN CAN GO CAN
   21: *MARY *JOHN *JOHN *BLAME *CAR *CAR *FUTURE CHICKEN            JOHN FISH WONT EAT BUT CAN EAT CHICKEN
   25: JOHN LIKE IX *LIKE IX                                         JOHN LIKE IX IX IX
   28: *ANN *ANN IX *MARY IX                                         JOHN LIKE IX IX IX
   30: *IX-1P *CHOCOLATE *MARY *LOVE *LOVE                           JOHN LIKE IX IX IX
   36: MARY *MARY *YESTERDAY *SHOOT LIKE *IX                         MARY VEGETABLE KNOW IX LIKE CORN1
   40: *MARY *JOHN *FUTURE1 *VEGETABLE *MARY                         JOHN IX THINK MARY LOVE
   43: JOHN *FUTURE BUY HOUSE       

**Question 3:**  Summarize the error results from three combinations of features and model selectors.  What was the "best" combination and why?  What additional information might we use to improve our WER?  For more insight on improving WER, take a look at the introduction to Part 4.

**Answer 3:** Out of the four features, feature_polar clearly performs the best and yields WER around 0.55 in all three model selectors. Norm is arguably the second best feature. Delta and custom features behaves quite similar. Among the 3 model selectors, their performances are quite similarly. The "best" combination is polar and DIC - polar takes into account the positions of both left and right hands relative to center (nose) while DIC could help to reduce model complexity. To improve WER, one thing is to include the context information in the combination of n-grams. 

<a id='part4_info'></a>
## PART 4: Improve the WER with Language Models
We've squeezed just about as much as we can out of the model and still only get about 50% of the words right! Surely we can do better than that.  Probability to the rescue again in the form of [statistical language models (SLM)](https://en.wikipedia.org/wiki/Language_model).  The basic idea is that each word has some probability of occurrence within the set, and some probability that it is adjacent to specific other words. We can use that additional information to make better choices.

##### Additional reading and resources
- [Introduction to N-grams (Stanford Jurafsky slides)](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf)
- [Speech Recognition Techniques for a Sign Language Recognition System, Philippe Dreuw et al](https://www-i6.informatik.rwth-aachen.de/publications/download/154/Dreuw--2007.pdf) see the improved results of applying LM on *this* data!
- [SLM data for *this* ASL dataset](ftp://wasserstoff.informatik.rwth-aachen.de/pub/rwth-boston-104/lm/)

##### Optional challenge
The recognizer implemented in Part 3 is equivalent to a "0-gram" SLM.  Here I improve the WER with the SLM data provided with the data set in the link above using "2-gram", and "3-gram" statistics, by using Viterbi like algorithm. 

In [63]:
# create a DataFrame of log likelihoods for the test word items
df_probs = pd.DataFrame(data=probabilities)
df_probs.head()

,ALL,ANN,APPLE,ARRIVE,BILL,BLAME,BLUE,BOOK,BORROW,BOX,...,VIDEOTAPE,VISIT,WANT,WHAT,WHO,WILL,WOMAN,WONT,WRITE,YESTERDAY
0,-2564.662695,-1.597319e+05,-2607.661913,-330.513567,-1917.200589,-361.877026,-2873.528478,-1038.955685,-4494.410087,-654.508016,...,-1568.623514,-166.193663,-2.308451e+05,-278.983071,-211.611880,-1266.658595,-738.568305,-754.633694,-12951.833687,-300.522945
1,-6791.690644,-1.892720e+05,-9809.478432,-118.958118,-20286.529893,-275.277465,-7970.543151,-113.149819,-3834.421244,-553.379536,...,-135.499407,-444.150906,-4.286967e+04,-341.874211,-111.098536,-4747.124223,-622.966748,-1494.193684,-113.951785,-596.329517
2,-9576.855303,-6.821695e+05,-16073.901762,-318.573229,-30641.026944,-659.456482,-11676.616479,-879.130768,-3441.069400,-1191.365385,...,-901.724914,-551.056907,-3.411478e+05,-760.664399,-483.978522,-7242.007128,-1652.088447,-1869.519352,-4587.090825,-962.369245
3,-1061.709747,-2.372305e+06,-2555.973123,-279.642514,-592.956086,-255.377577,-567.224467,-1171.290246,-33897.899104,-690.245418,...,-3942.972597,-541.122128,-1.143249e+06,-704.807704,-565.158810,-11861.278942,-194.551810,-508.134793,-118135.870836,-521.070155
4,-1462.548501,-6.572611e+05,-3113.233070,-99.786725,-3225.166830,-57.323454,-638.972883,-164.498017,-846.567498,-49.611306,...,-231.166543,-64.713586,-4.565087e+05,-50.739092,-208.252232,-5502.558401,-194.677316,-812.568059,-6361.406513,-486.300055


In [94]:
Xlengths = test_set.get_all_Xlengths()
conv = np.log10(np.exp(1.))

In [95]:
import operator
import arpa
model_lm = arpa.loadf("ukn.3.lm")
lm = model_lm[0]  # ARPA files may contain several models.

### 2-gram

In [316]:
# hmm + 2-gram slm model
# implemented by Viterbi algo.

def lm_search2(lm_scale=1.):
    
    '''search with given lm_scale.'''
    
# keep the top 10 in each hmm states 

    guess_viterbi = []

    # iterate through sequences by key in test_set

    for key in test_set.sentences_index:

        result = []

        seq = seq = {'<s>':0}
        result.append(seq)

        for v in test_set.sentences_index[key]:
    
            curr_states = {}
    
            last_states = result[-1]
    
            # top 10 hmm states from original df_probs
            hmm_state = dict(sorted(df_probs.ix[v].items(), key=operator.itemgetter(1), reverse=True)[:10])
        
            for w in hmm_state:
        
            # phi_j (t)
        
                loghmm = hmm_state[w]

                cur_max = float('-inf')
                cur_state = None
        
            # phi_i(t-1)
        
                for state in last_states:            
        
                    last_max = last_states[state]
            
                    last_word = state.split()[-1]
            
                # 2-gram
                    words = last_word + " " + w
            
                # a_ij
                    try: # conditional P(w | w)
                
                        logslm = lm.log_p(words)/conv
            
                    except: # not found in arpa, use default -99.
                
                        logslm = -99./conv

                    cur_val = last_max + lm_scale * logslm
            
                    #print (w, state, last_max, logslm)
            
                    if cur_val > cur_max:
                        cur_state = state + " " + w
                        cur_max = cur_val
                
                curr_states[cur_state] = loghmm + cur_max
        
            result.append(curr_states)
    
        # sentence ending: </s>

        curr_states = {}    
        last_states = result[-1]

        cur_max = float('-inf')
        cur_state = None
        
        # phi_i(t-1)
        
        for state in last_states:            
        
            last_max = last_states[state]
            
            last_word = state.split()[-1]
            
        # 2-gram
            words = last_word + " </s>" 
            
        # a_ij
            try: # conditional P(w | w)
                
                logslm = lm.log_p(words)/conv
            
            except: # not found in arpa, use default -99.
                
                logslm = -99./conv

            cur_val = last_max + lm_scale * logslm
            
            #print (w, state, last_max, logslm)
            
            if cur_val > cur_max:
                cur_state = state + " </s>"
                cur_max = cur_val
                
        curr_states[cur_state] = cur_max
        
        result.append(curr_states)
    
        guess_viterbi.extend(list(result[-1].keys())[0].split()[1:-1])

    return show_errors_2(guess_viterbi, test_set)



In [307]:
# use df_probs
for i in range(1, 100, 10):
    print (i, lm_search2(i))

1 0.5337078651685393
11 0.4887640449438202
21 0.5056179775280899
31 0.5
41 0.5393258426966292
51 0.550561797752809
61 0.550561797752809
71 0.5617977528089888
81 0.6179775280898876
91 0.6235955056179775


### 3-gram

In [321]:
# hmm + 3-gram slm model
# implemented by Viterbi algo.

def lm_search3(lm_scale=1.):
    
    '''search with given lm_scale.'''
    
# keep the top 10 in each hmm states 

    guess_viterbi = []

    # iterate through sequences by key in test_set

    for key in test_set.sentences_index:

        result = []

        seq = seq = {'<s>':0}
        result.append(seq)

        for v in test_set.sentences_index[key]:
    
            curr_states = {}
    
            last_states = result[-1]
    
            # top 10 hmm states from original df_probs
            hmm_state = dict(sorted(df_probs.ix[v].items(), key=operator.itemgetter(1), reverse=True)[:10])
        
            for w in hmm_state:
        
            # phi_j (t)
        
                loghmm = hmm_state[w]

                cur_max = float('-inf')
                cur_state = None
        
            # phi_i(t-1)
        
                for state in last_states:            
        
                    last_max = last_states[state]
            
                    last_word = " ".join(state.split()[-2:])
            
                # 2-gram
                    words = last_word + " " + w
            
                # a_ij
                    try: # conditional P(w | w)
                
                        logslm = lm.log_p(words)/conv
            
                    except: # not found in arpa, use default -99.
                
                        logslm = -99./conv

                    cur_val = last_max + lm_scale * logslm
            
                    #print (w, state, last_max, logslm)
            
                    if cur_val > cur_max:
                        cur_state = state + " " + w
                        cur_max = cur_val
                
                curr_states[cur_state] = loghmm + cur_max
        
            result.append(curr_states)
    
        # sentence ending: </s>

        curr_states = {}    
        last_states = result[-1]

        cur_max = float('-inf')
        cur_state = None
        
        # phi_i(t-1)
        
        for state in last_states:            
        
            last_max = last_states[state]
            
            last_word = " ".join(state.split()[-2:])
            
        # 2-gram
            words = last_word + " </s>" 
            
        # a_ij
            try: # conditional P(w | w)
                
                logslm = lm.log_p(words)/conv
            
            except: # not found in arpa, use default -99.
                
                logslm = -99./conv

            cur_val = last_max + lm_scale * logslm
            
            #print (w, state, last_max, logslm)
            
            if cur_val > cur_max:
                cur_state = state + " </s>"
                cur_max = cur_val
                
        curr_states[cur_state] = cur_max
        
        result.append(curr_states)
    
        guess_viterbi.extend(list(result[-1].keys())[0].split()[1:-1])

    return show_errors_2(guess_viterbi, test_set)



In [320]:
# use df_probs
for i in range(1, 100, 10):
    print (i, lm_search3(i))

1 0.5224719101123596
11 0.449438202247191
21 0.39325842696629215
31 0.4157303370786517
41 0.43258426966292135
51 0.4606741573033708
61 0.4606741573033708
71 0.47191011235955055
81 0.48314606741573035
91 0.48314606741573035


### Summary

Implemented SLM with HMM in Viterbi algorithm. The HMM probabilities are the probabilites of observation in each state (word); SLM probabilities from arpa library are the transition probabilities between the states. Top 10 HMM probabilities are kept in the search to expedite the process. The execution time of Viterbi algorithm for given LM scale parameter is ~ 10 seconds in my environment.

a) original HMM with features_polar (BIC as score function) alone has a WER of 55%.

b) 2-gram SLM with HMM achieves a WER of 49%.

c) 3-gram SLM with HMM achieves a WER of 39%.